In [1]:
#Baseline Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_log_error as msle

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
train_data = pd.read_csv('updated_dataset/updated_train_data.csv')
product_data = pd.read_csv('updated_dataset/updated_product_data.csv')
store_data = pd.read_csv('updated_dataset/updated_store_data.csv')

In [3]:
train_data.head()

,WEEK_END_DATE,STORE_NUM,UPC,BASE_PRICE,FEATURE,DISPLAY,UNITS
0,14-Jan-09,367,1111009477,1.57,0,0,13
1,14-Jan-09,367,1111009497,1.39,0,0,20
2,14-Jan-09,367,1111085319,1.88,0,0,14
3,14-Jan-09,367,1111085345,1.88,0,0,29
4,14-Jan-09,367,1111085350,1.98,0,0,35


In [4]:
product_data.head()

,UPC,MANUFACTURER_1,MANUFACTURER_2,MANUFACTURER_3,MANUFACTURER_4,MANUFACTURER_5,MANUFACTURER_6,MANUFACTURER_7,MANUFACTURER_8,MANUFACTURER_9,...,CATEGORY_3,CATEGORY_4,SUB_CATEGORY_1,SUB_CATEGORY_2,SUB_CATEGORY_3,SUB_CATEGORY_4,SUB_CATEGORY_5,SUB_CATEGORY_6,SUB_CATEGORY_7,PRODUCT_SIZE
0,1111009477,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
1,1111009497,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
2,1111009507,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2
3,1111038078,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1111038080,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [5]:
store_data.head()

,STORE_ID,ADDRESS_STATE_PROV_CODE_1,ADDRESS_STATE_PROV_CODE_2,ADDRESS_STATE_PROV_CODE_3,ADDRESS_STATE_PROV_CODE_4,MSA_CODE_1,MSA_CODE_2,MSA_CODE_3,MSA_CODE_4,MSA_CODE_5,MSA_CODE_6,MSA_CODE_7,MSA_CODE_8,MSA_CODE_9,SEG_VALUE_NAME,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,367,1,0,0,0,1,0,0,0,0,0,0,0,0,1,24721,12707
1,389,1,0,0,0,1,0,0,0,0,0,0,0,0,2,46073,24767
2,613,0,1,0,0,0,1,0,0,0,0,0,0,0,2,64926,29386
3,623,0,1,0,0,0,0,1,0,0,0,0,0,0,2,46930,36741
4,2277,0,0,1,0,1,0,0,0,0,0,0,0,0,3,81958,54053


### Merge Tables

In [6]:
# merge the datasets
merge_data = train_data.merge(product_data, how='left', on='UPC')
merge_data = merge_data.merge(store_data, how='left', left_on='STORE_NUM', right_on='STORE_ID')

In [7]:
merge_data = merge_data.drop(columns=['STORE_ID'])

In [8]:
merge_data.isna().sum().sum()

0

In [9]:
merge_data.loc[0]

WEEK_END_DATE                 14-Jan-09
STORE_NUM                           367
UPC                          1111009477
BASE_PRICE                         1.57
FEATURE                               0
DISPLAY                               0
UNITS                                13
MANUFACTURER_1                        1
MANUFACTURER_2                        0
MANUFACTURER_3                        0
MANUFACTURER_4                        0
MANUFACTURER_5                        0
MANUFACTURER_6                        0
MANUFACTURER_7                        0
MANUFACTURER_8                        0
MANUFACTURER_9                        0
CATEGORY_1                            1
CATEGORY_2                            0
CATEGORY_3                            0
CATEGORY_4                            0
SUB_CATEGORY_1                        1
SUB_CATEGORY_2                        0
SUB_CATEGORY_3                        0
SUB_CATEGORY_4                        0
SUB_CATEGORY_5                        0


### Creating the Validation Set

In [10]:
# convert the column WEEK_END_DATE to datetime format
merge_data.WEEK_END_DATE = pd.to_datetime(merge_data.WEEK_END_DATE)

In [11]:
# store the unique dates 
weeks = merge_data.WEEK_END_DATE.unique()

In [12]:
# define the function that will return a dictionary which contains the keys

def get_train_validation_set(number=1):
    validation_sets = []
    for n in range(number):
        x={}
        
        x['validation_set'] = weeks[len(weeks)-n-1]
        x['train_set_end_date'] = weeks[len(weeks)-n-3]
        validation_sets.append(x)
    return validation_sets

In [13]:
validation_sets = get_train_validation_set(number=5)

In [14]:
validation_sets

[{'validation_set': numpy.datetime64('2011-09-28T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2011-09-14T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-21T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2011-09-07T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-14T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2011-08-31T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-09-07T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2011-08-24T00:00:00.000000000')},
 {'validation_set': numpy.datetime64('2011-08-31T00:00:00.000000000'),
  'train_set_end_date': numpy.datetime64('2011-08-17T00:00:00.000000000')}]

In [15]:
data_set = []

for data in validation_sets:
    training_data = merge_data[merge_data.WEEK_END_DATE <= data['train_set_end_date']]
    validation_data = merge_data[merge_data.WEEK_END_DATE == data['validation_set']]
    
    data_set.append((training_data, validation_data))

### Mean Prediction



In [16]:
# define the function to get the RMSLE

def get_msle(true, predicted):
    return np.sqrt(msle(true,predicted))

In [23]:
train_rmsle = []
valid_rmsle = []

for i, data in enumerate(data_set):
    # get the train and validation set
    train, valid = data
    
    # get the mean prediction dataframe by using a groupby on STORE_NUM and UPC
    mean_prediction = train.groupby(['STORE_NUM', 'UPC'])['UNITS'].mean().reset_index()
    
    # left join the train and validation set with the mean prediction
    train = train.merge(mean_prediction, how='left', on=['STORE_NUM', 'UPC'])
    valid = valid.merge(mean_prediction, how='left', on=['STORE_NUM', 'UPC'])
    
    # get the RMSLE on train and validation set
    t_rmsle = get_msle(train.UNITS_x, train.UNITS_y)
    v_rmsle = get_msle(valid.UNITS_x, valid.UNITS_y)
    train_rmsle.append(t_rmsle)
    valid_rmsle.append(v_rmsle)
    
    print("RMSLE ON TRAINING SET: ", i+1, ": ", t_rmsle)
    print("RMSLE ON VALIDATION SET: ", i+1, ": ", v_rmsle)
    print("====================================================================")
    
# get the mean RMSLE on train and validation set
print("Mean RMSLE ON TRAIN: ", np.mean(train_rmsle))
print("Mean RMSLE ON VALID: ", np.mean(valid_rmsle))

RMSLE ON TRAINING SET:  1 :  0.5902592110738579
RMSLE ON VALIDATION SET:  1 :  0.5887804241752373
RMSLE ON TRAINING SET:  2 :  0.5912639141033686
RMSLE ON VALIDATION SET:  2 :  0.6263169979832923
RMSLE ON TRAINING SET:  3 :  0.5917964778574165
RMSLE ON VALIDATION SET:  3 :  0.4783767090098456
RMSLE ON TRAINING SET:  4 :  0.5914356263311358
RMSLE ON VALIDATION SET:  4 :  0.5811810487862565
RMSLE ON TRAINING SET:  5 :  0.5916390592275275
RMSLE ON VALIDATION SET:  5 :  0.7181642414489362
Mean RMSLE ON TRAIN:  0.5912788577186613
Mean RMSLE ON VALID:  0.5985638842807136


### Simple Moving Average

In [24]:
def get_sma(i, train, valid, no_of_weeks=2):
    
    # create a copy of train and validation set
    train_copy = train.copy()
    valid_copy = valid.copy()
    
    # group the data by store-num and upc and use rolling and mean function to calculate the moving average
    data_copy = train_copy.groupby(['STORE_NUM', 'UPC'])['UNITS'].rolling(no_of_weeks).mean().reset_index().set_index('WEEK_END_DATE')
    
    # add the moving avg column to the train data
    train_copy['moving_average'] = data_copy['UNITS']
    
    last_average= train_copy.groupby(['STORE_NUM', 'UPC'])['moving_average'].last().reset_index()
    
    train_copy = train_copy[['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS', 'moving_average']]
    valid_copy = valid_copy[['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS']]
    
    train_copy.dropna(inplace=True)
    valid_copy = valid_copy.merge(last_average, how='left', on=['STORE_NUM', 'UPC'])
    
    # calculate the rmsle on train and validation set
    t_rmsle = get_msle(train_copy['UNITS'], train_copy['moving_average'])
    v_rmsle = get_msle(valid_copy['UNITS'], valid_copy['moving_average'])
    
    print("RMSLE ON TRAINING SET: ",i+1, ': ', t_rmsle)
    print("RMSLE ON VALIDATION SET: ",i+1, ': ', v_rmsle)
    print("=======================================================================")
    
    return t_rmsle, v_rmsle

train_rmsle_ma = []
valid_rmsle_ma = []

for i, data in enumerate(data_set):
    train, valid = data
    
    t_rmsle, v_rmsle = get_sma(i, train, valid, no_of_weeks=8)
    train_rmsle_ma.append(t_rmsle)
    valid_rmsle_ma.append(v_rmsle)
    
print('Mean RMSLE on Train: ', np.mean(train_rmsle_ma))
print('Mean RMSLE on Valid: ', np.mean(valid_rmsle_ma))

### Linear Regression

In [26]:
train_rmsle_lr = []
valid_rmsle_lr = []

for i, data in enumerate(data_set):
    # get the train and validation set
    train, valid = data
    
    # left join the train and validation set with the mean prediction
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    model_LR = LinearRegression(normalize=True)
    model_LR.fit(train_x, train_y)
    
    predict_train = model_LR.predict(train_x).clip(min=0)
    predict_valid = model_LR.predict(valid_x).clip(min=0)
    
    # get the RMSLE on train and validation set
    t_rmsle = get_msle(train_y,predict_train)
    v_rmsle = get_msle(valid_y,predict_valid)
    train_rmsle.append(t_rmsle)
    valid_rmsle.append(v_rmsle)
    
    print("RMSLE ON TRAINING SET: ", i+1, ": ", t_rmsle)
    print("RMSLE ON VALIDATION SET: ", i+1, ": ", v_rmsle)
    print("====================================================================")
    
# get the mean RMSLE on train and validation set
print("Mean RMSLE ON TRAIN: ", np.mean(train_rmsle))
print("Mean RMSLE ON VALID: ", np.mean(valid_rmsle))

RMSLE ON TRAINING SET:  1 :  0.9955444824233143
RMSLE ON VALIDATION SET:  1 :  0.9213076621924107
RMSLE ON TRAINING SET:  2 :  0.9911388560853901
RMSLE ON VALIDATION SET:  2 :  0.8995541069643282
RMSLE ON TRAINING SET:  3 :  0.9913968630882163
RMSLE ON VALIDATION SET:  3 :  0.962818370955821
RMSLE ON TRAINING SET:  4 :  0.9926915214574428
RMSLE ON VALIDATION SET:  4 :  0.9546727827002203
RMSLE ON TRAINING SET:  5 :  0.9953905649418425
RMSLE ON VALIDATION SET:  5 :  0.9765950756678158
Mean RMSLE ON TRAIN:  0.7922556576589513
Mean RMSLE ON VALID:  0.7707767419884164


### Decision Tree 

In [27]:
train_rmsle_dtr = []
valid_rmsle_dtr = []

for i, data in enumerate(data_set):
    # get the train and validation set
    train, valid = data
    
    # left join the train and validation set with the mean prediction
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    model_DTR = DecisionTreeRegressor()
    model_DTR.fit(train_x, train_y)
    
    predict_train = model_DTR.predict(train_x).clip(min=0)
    predict_valid = model_DTR.predict(valid_x).clip(min=0)
    
    # get the RMSLE on train and validation set
    t_rmsle = get_msle(train_y,predict_train)
    v_rmsle = get_msle(valid_y,predict_valid)
    train_rmsle_dtr.append(t_rmsle)
    valid_rmsle_dtr.append(v_rmsle)
    
    print("RMSLE ON TRAINING SET: ", i+1, ": ", t_rmsle)
    print("RMSLE ON VALIDATION SET: ", i+1, ": ", v_rmsle)
    print("====================================================================")
    
# get the mean RMSLE on train and validation set
print("Mean RMSLE ON TRAIN: ", np.mean(train_rmsle_dtr))
print("Mean RMSLE ON VALID: ", np.mean(valid_rmsle_dtr))

RMSLE ON TRAINING SET:  1 :  0.41667563384749384
RMSLE ON VALIDATION SET:  1 :  0.45421680306271534
RMSLE ON TRAINING SET:  2 :  0.41663996700336603
RMSLE ON VALIDATION SET:  2 :  0.49539971404073285
RMSLE ON TRAINING SET:  3 :  0.41635554332551844
RMSLE ON VALIDATION SET:  3 :  0.459741392987353
RMSLE ON TRAINING SET:  4 :  0.4159250978004229
RMSLE ON VALIDATION SET:  4 :  0.5177879394330758
RMSLE ON TRAINING SET:  5 :  0.4158593590916643
RMSLE ON VALIDATION SET:  5 :  0.588127174049146
Mean RMSLE ON TRAIN:  0.41629112021369313
Mean RMSLE ON VALID:  0.5030546047146046


### Random Forest

In [28]:
train_rmsle_dtr = []
valid_rmsle_dtr = []

for i, data in enumerate(data_set):
    # get the train and validation set
    train, valid = data
    
    # left join the train and validation set with the mean prediction
    train_x = train.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    train_y = train['UNITS']
    
    valid_x = valid.drop(columns=['WEEK_END_DATE', 'STORE_NUM', 'UPC', 'UNITS'])
    valid_y = valid['UNITS']
    
    model_DTR = RandomForestRegressor(random_state=0)
    model_DTR.fit(train_x, train_y)
    
    predict_train = model_DTR.predict(train_x).clip(min=0)
    predict_valid = model_DTR.predict(valid_x).clip(min=0)
    
    # get the RMSLE on train and validation set
    t_rmsle = get_msle(train_y,predict_train)
    v_rmsle = get_msle(valid_y,predict_valid)
    train_rmsle_dtr.append(t_rmsle)
    valid_rmsle_dtr.append(v_rmsle)
    
    print("RMSLE ON TRAINING SET: ", i+1, ": ", t_rmsle)
    print("RMSLE ON VALIDATION SET: ", i+1, ": ", v_rmsle)
    print("====================================================================")
    
# get the mean RMSLE on train and validation set
print("Mean RMSLE ON TRAIN: ", np.mean(train_rmsle_dtr))
print("Mean RMSLE ON VALID: ", np.mean(valid_rmsle_dtr))

RMSLE ON TRAINING SET:  1 :  0.4217605462439632
RMSLE ON VALIDATION SET:  1 :  0.4391385693265205
RMSLE ON TRAINING SET:  2 :  0.42174972277437306
RMSLE ON VALIDATION SET:  2 :  0.46793496690839687
RMSLE ON TRAINING SET:  3 :  0.42146623952292034
RMSLE ON VALIDATION SET:  3 :  0.4437868022855753
RMSLE ON TRAINING SET:  4 :  0.42102647939541493
RMSLE ON VALIDATION SET:  4 :  0.5089890017503047
RMSLE ON TRAINING SET:  5 :  0.42095942857460167
RMSLE ON VALIDATION SET:  5 :  0.5711438943066157
Mean RMSLE ON TRAIN:  0.42139248330225454
Mean RMSLE ON VALID:  0.4861986469154826
